## Задание 2: Работа с датасетами (30 баллов)

### 2.1 Кастомный Dataset класс (15 баллов)
```python
# Создайте кастомный класс датасета для работы с CSV файлами:
# - Загрузка данных из файла
# - Предобработка (нормализация, кодирование категорий)
# - Поддержка различных форматов данных (категориальные, числовые, бинарные и т.д.)
```

In [41]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [56]:
class CustomDataset(Dataset):
    def __init__(self, path_file, numeric_cols, categorical_cols, binary_cols, target_col):
        self.df = pd.read_csv(path_file)
        self.path_file = path_file
        
        # Заполнение пропусков — исправлено без inplace
        for col in numeric_cols:
            self.df[col] = self.df[col].fillna(self.df[col].median())
        for col in categorical_cols:
            self.df[col] = self.df[col].fillna(self.df[col].mode()[0])
        for col in binary_cols:
            self.df[col] = self.df[col].fillna(0)

        # Обработка числовых признаков
        self.scaler = StandardScaler()
        numeric_data = self.scaler.fit_transform(self.df[numeric_cols])

        # Обработка категориальных признаков
        try:
            self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        except TypeError:
            self.encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

        cat_data = self.encoder.fit_transform(self.df[categorical_cols])
        cat_data = np.array(cat_data)

        # Обработка бинарных признаков
        if binary_cols:
            binary_data = self.df[binary_cols].astype(int).values
        else:
            binary_data = np.empty((len(self.df), 0))

        # Обработка целевой переменной
        y_raw = self.df[target_col]
        if y_raw.dtype == 'object':
            self.label_encoder = LabelEncoder()
            y = self.label_encoder.fit_transform(y_raw)
        else:
            y = y_raw.values
        self.y = torch.tensor(y, dtype=torch.float32)

        # Объединение всех признаков
        X = np.concatenate([numeric_data, cat_data, binary_data], axis=1)
        self.X = torch.tensor(X, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### 2.2 Эксперименты с различными датасетами (15 баллов)
```python
# Найдите csv датасеты для регрессии и бинарной классификации и, применяя наработки из предыдущей части задания, обучите линейную и логистическую регрессию
```

In [53]:
import torch
from torch.utils.data import DataLoader
from linRegAndLogReg import LinearRegression, LogisticRegression

In [58]:
titanic_dataset = CustomDataset(
        path_file='train.csv',
        numeric_cols=['Age', 'Fare'],
        categorical_cols=['Pclass', 'Sex', 'Embarked'],
        binary_cols=[],
        target_col='Survived'
)

spotify_dataset = CustomDataset(
    path_file='spotify.csv',
    numeric_cols=['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
                    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
    categorical_cols=['mode', 'key', 'time_signature', 'track_genre'],
    binary_cols=['explicit'],
    target_col='popularity'
)

In [60]:
def train_model(model, dataloader, criterion, optimizer, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


def fit_model(dataset, batch_size, model, criterion, optimizer):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print(f"Training {str(model).split('(')[0]} on {dataset.path_file}")
    train_model(model, dataloader, criterion, optimizer)


if __name__ == '__main__':
    model = LogisticRegression(in_features = titanic_dataset.X.shape[1])
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    fit_model(titanic_dataset, 32, model, criterion, optimizer)
    
    model = LinearRegression(in_features = spotify_dataset.X.shape[1])
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    fit_model(spotify_dataset, 32, model, criterion, optimizer)

Training LogisticRegression on train.csv
Epoch 10/50, Loss: 0.4558
Epoch 20/50, Loss: 0.4510
Epoch 30/50, Loss: 0.4498
Epoch 40/50, Loss: 0.4484
Epoch 50/50, Loss: 0.4494
Training LinearRegression on spotify.csv
Epoch 10/50, Loss: 379.0816
Epoch 20/50, Loss: 378.7777
Epoch 30/50, Loss: 379.1114
Epoch 40/50, Loss: 379.3979
Epoch 50/50, Loss: 378.9909
